In [1]:
import pandas as pd
import numpy as np
from tree import AFTSurvivalTree
from forest import AFTForest
from dataset import SupportDataset, SyntheticDataset, VeteranLungDataset
from sklearn.model_selection import train_test_split
import time

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv('../data/veterans_lung_cancer.csv')
df.describe()

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Survival_label_lower_bound,Survival_label_upper_bound,Age_in_years,Karnofsky_score,Months_from_Diagnosis,Celltype=adeno,Celltype=large,Celltype=smallcell,Celltype=squamous,Prior_therapy=no,Prior_therapy=yes,Treatment=standard,Treatment=test
count,137.000000,137.0,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000
mean,121.627737,inf,58.306569,58.569343,8.773723,0.197080,0.197080,0.350365,0.255474,0.708029,0.291971,0.503650,0.496350
std,157.816722,NaN,10.541628,20.039592,10.612141,0.399253,0.399253,0.478835,0.437728,0.456337,0.456337,0.501821,0.501821
min,1.000000,1.0,34.000000,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,25.0,51.000000,40.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,80.000000,80.0,62.000000,60.000000,5.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,144.000000,177.0,66.000000,75.000000,11.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,999.000000,inf,81.000000,99.000000,87.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
df

,Survival_label_lower_bound,Survival_label_upper_bound,Age_in_years,Karnofsky_score,Months_from_Diagnosis,Celltype=adeno,Celltype=large,Celltype=smallcell,Celltype=squamous,Prior_therapy=no,Prior_therapy=yes,Treatment=standard,Treatment=test
0,72.0,72.0,69.0,60.0,7.0,0,0,0,1,1,0,1,0
1,411.0,411.0,64.0,70.0,5.0,0,0,0,1,0,1,1,0
2,228.0,228.0,38.0,60.0,3.0,0,0,0,1,1,0,1,0
3,126.0,126.0,63.0,60.0,9.0,0,0,0,1,0,1,1,0
4,118.0,118.0,65.0,70.0,11.0,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,133.0,133.0,65.0,75.0,1.0,0,1,0,0,1,0,0,1
133,111.0,111.0,64.0,60.0,5.0,0,1,0,0,1,0,0,1
134,231.0,231.0,67.0,70.0,18.0,0,1,0,0,0,1,0,1
135,378.0,378.0,65.0,80.0,4.0,0,1,0,0,1,0,0,1


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, genextreme, gaussian_kde, ecdf

In [5]:
veteran = VeteranLungDataset(df)
veteran.get_data()

array([[69., 60.,  7., ...,  0.,  1.,  0.],
       [64., 70.,  5., ...,  1.,  1.,  0.],
       [38., 60.,  3., ...,  0.,  1.,  0.],
       ...,
       [67., 70., 18., ...,  1.,  0.,  1.],
       [65., 80.,  4., ...,  0.,  0.,  1.],
       [37., 30.,  3., ...,  0.,  0.,  1.]])

In [6]:
veteran.get_label()

array([(0,  72.), (0, 411.), (0, 228.), (0, 126.), (0, 118.), (0,  10.),
       (0,  82.), (0, 110.), (0, 314.), (1, 100.), (0,  42.), (0,   8.),
       (0, 144.), (1,  25.), (0,  11.), (0,  30.), (0, 384.), (0,   4.),
       (0,  54.), (0,  13.), (1, 123.), (1,  97.), (0, 153.), (0,  59.),
       (0, 117.), (0,  16.), (0, 151.), (0,  22.), (0,  56.), (0,  21.),
       (0,  18.), (0, 139.), (0,  20.), (0,  31.), (0,  52.), (0, 287.),
       (0,  18.), (0,  51.), (0, 122.), (0,  27.), (0,  54.), (0,   7.),
       (0,  63.), (0, 392.), (0,  10.), (0,   8.), (0,  92.), (0,  35.),
       (0, 117.), (0, 132.), (0,  12.), (0, 162.), (0,   3.), (0,  95.),
       (0, 177.), (0, 162.), (0, 216.), (0, 553.), (0, 278.), (0,  12.),
       (0, 260.), (0, 200.), (0, 156.), (1, 182.), (0, 143.), (0, 105.),
       (0, 103.), (0, 250.), (0, 100.), (0, 999.), (0, 112.), (1,  87.),
       (1, 231.), (0, 242.), (0, 991.), (0, 111.), (0,   1.), (0, 587.),
       (0, 389.), (0,  33.), (0,  25.), (0, 357.), 

In [7]:
veteran.get_xgboost_label()

,Survival_label_lower_bound,Survival_label_upper_bound
0,72.0,72.0
1,411.0,411.0
2,228.0,228.0
3,126.0,126.0
4,118.0,118.0
...,...,...
132,133.0,133.0
133,111.0,111.0
134,231.0,231.0
135,378.0,378.0


In [8]:
X_train, X_test, y_train, y_test = veteran.get_train_test()

##### Init All Methods

AFT-Forest

In [9]:
aft_surv_tree = AFTSurvivalTree(function="norm")
aft_forest = AFTForest(n_trees=10, random_params=True)

In [10]:
aft_surv_tree.fit(X_train, y_train)

In [11]:
aft_forest.fit(X_train, y_train)

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/m

In [12]:
aft_surv_tree.predict(X_test)

[125.5,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 70.2,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 70.2,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 104.1,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 104.1,
 127.28915662650603]

In [13]:
aft_forest.predict(X_test)

[103.44051288749563,
 100.48023510971787,
 97.75582674030949,
 128.9818007662835,
 133.51870552818826,
 110.72099331286738,
 112.13276230474506,
 114.4497661342489,
 116.5457168501996,
 109.72571685019962,
 101.32488220175625,
 103.44051288749563,
 137.15054313502588,
 137.15054313502588,
 116.5457168501996,
 126.99103604791011,
 121.29526230474505,
 116.2883178603006,
 135.93844412292688,
 129.1472265241006,
 127.57099331286736,
 135.93844412292688,
 116.2883178603006,
 99.72430076628352,
 133.51870552818826,
 107.73276230474505,
 110.72099331286738,
 109.72571685019962]

In [14]:
aft_surv_tree._score(X_test, y_test)

0.5206043956043956

In [15]:
aft_surv_tree._visualize()

In [16]:
aft_forest._score(X_test, y_test)

0.6456043956043956

In [17]:
aft_surv_tree_weibull = AFTSurvivalTree(function="weibull", is_bootstrap=False, is_custom_dist=True)


In [18]:
aft_surv_tree_weibull.fit(X_train, y_train)

In [19]:

aft_surv_tree_weibull._score(X_test, y_test)

0.5

In [20]:
aft_surv_tree_weibull.save("tree_weibull.json")

In [21]:
aft_surv_tree_load_test = AFTSurvivalTree()
aft_surv_tree_load_test = aft_surv_tree_load_test.load("tree_weibull.json")

In [22]:
aft_surv_tree_load_test._score(X_test, y_test)

0.5

In [23]:
aft_surv_tree_logistic = AFTSurvivalTree(function="logistic", is_bootstrap=False, is_custom_dist=True)

In [24]:
aft_surv_tree_logistic.fit(X_train, y_train)

In [25]:
aft_surv_tree_logistic._score(X_test, y_test)

0.5

In [26]:
aft_surv_tree_gmm = AFTSurvivalTree(function="gmm", is_bootstrap=False, is_custom_dist=True)

In [27]:
aft_surv_tree_gmm.fit(X_train, y_train)

In [28]:
aft_surv_tree_gmm._score(X_test, y_test)

0.5480769230769231

In [29]:
aft_surv_tree_normal = AFTSurvivalTree(function="norm", is_bootstrap=False, is_custom_dist=True)

In [30]:
aft_surv_tree_normal.fit(X_train, y_train)


In [31]:
aft_surv_tree_normal._score(X_test, y_test)

0.5

In [32]:
aft_surv_tree_extreme = AFTSurvivalTree(function="extreme", is_bootstrap=True, is_custom_dist=True)

In [33]:
aft_surv_tree_extreme.fit(X_train, y_train)

In [34]:
aft_surv_tree_extreme._score(X_test, y_test)

0.6291208791208791

In [35]:
aft_surv_tree_extreme._visualize()

In [36]:
aft_surv_tree_extreme.save("extreme.json")

In [37]:
kwargs = {
    "function": "gmm", 
    "is_bootstrap": False, 
    "is_custom_dist": True,
    "n_components": 2
}
aft_forest = AFTForest(n_trees=5, random_params=False, **kwargs)

In [38]:
aft_forest.fit(X_train, y_train)

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
100%|██████████| 5/5 [00:00<00:00, 15087.42it/s]


In [39]:
aft_forest._score(X_test, y_test)

0.6648351648351648

In [40]:
aft_forest.save("forest.json")

In [41]:
aft_forest_load_test = AFTForest()
aft_forest_load_test = aft_forest_load_test.load("forest.json")

In [42]:
# aft_forest_load_test._score(X_test, y_test)

In [43]:
# aft_forest_load_test.save("forest_load.json")

XGBoost-AFT

In [44]:
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit


In [45]:
X_train, X_test, y_train, y_test = veteran.get_train_test_xgboost()

X_train, X_test, y_train, y_test


(array([[81., 60.,  4., ...,  0.,  1.,  0.],
        [49., 60., 11., ...,  1.,  0.,  1.],
        [64., 80.,  5., ...,  0.,  1.,  0.],
        ...,
        [48., 70., 11., ...,  1.,  1.,  0.],
        [62., 30.,  2., ...,  0.,  0.,  1.],
        [70., 70.,  2., ...,  0.,  0.,  1.]]),
 array([[59., 30., 87.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.],
        [71., 50., 17.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.],
        [63., 30.,  4.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.],
        [69., 50., 12.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.],
        [69., 40.,  3.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.],
        [38., 80.,  5.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.],
        [60., 90.,  8.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.],
        [66., 50.,  5.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.],
        [64., 60.,  5.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.],
        [63., 70., 11.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.],
        [35., 50.,  7.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.],


In [46]:

dtrain = xgb.DMatrix(X_train)
dtrain.set_float_info('label_lower_bound', y_train['Survival_label_lower_bound'])
dtrain.set_float_info('label_upper_bound', y_train['Survival_label_upper_bound'])
dvalid = xgb.DMatrix(X_test)
dvalid.set_float_info('label_lower_bound', y_test['Survival_label_lower_bound'])
dvalid.set_float_info('label_upper_bound', y_test['Survival_label_upper_bound'])


In [47]:


params = {'verbosity': 0,
          'objective': 'survival:aft',
          'eval_metric': 'aft-nloglik',
          'tree_method': 'hist',
          'learning_rate': 0.05,
          'aft_loss_distribution': 'normal',
          'aft_loss_distribution_scale': 1.20,
          'max_depth': 6,
          'lambda': 0.01,
          'alpha': 0.02}

bst = xgb.train(params, dtrain, num_boost_round=10000,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                early_stopping_rounds=50)


[0]	train-aft-nloglik:12.65931	valid-aft-nloglik:12.95408
[1]	train-aft-nloglik:11.91142	valid-aft-nloglik:12.23321
[2]	train-aft-nloglik:11.23593	valid-aft-nloglik:11.57901
[3]	train-aft-nloglik:10.62571	valid-aft-nloglik:10.99612
[4]	train-aft-nloglik:10.07451	valid-aft-nloglik:10.46704
[5]	train-aft-nloglik:9.57661	valid-aft-nloglik:9.98793
[6]	train-aft-nloglik:9.12682	valid-aft-nloglik:9.56821
[7]	train-aft-nloglik:8.71970	valid-aft-nloglik:9.19213
[8]	train-aft-nloglik:8.35124	valid-aft-nloglik:8.85090
[9]	train-aft-nloglik:8.01783	valid-aft-nloglik:8.53140
[10]	train-aft-nloglik:7.71686	valid-aft-nloglik:8.25658
[11]	train-aft-nloglik:7.44448	valid-aft-nloglik:8.00365
[12]	train-aft-nloglik:7.19820	valid-aft-nloglik:7.78226
[13]	train-aft-nloglik:6.97602	valid-aft-nloglik:7.58773
[14]	train-aft-nloglik:6.77291	valid-aft-nloglik:7.41240
[15]	train-aft-nloglik:6.59171	valid-aft-nloglik:7.24455
[16]	train-aft-nloglik:6.42660	valid-aft-nloglik:7.10553
[17]	train-aft-nloglik:6.27758	

In [48]:
times_pred = bst.predict(dvalid, output_margin=True)
times_pred

array([0.78781796, 3.8089917 , 4.091513  , 2.408381  , 3.997563  ,
       5.625654  , 5.819438  , 2.7814074 , 5.416122  , 4.2301164 ,
       5.428511  , 2.5887406 , 5.06476   , 5.721085  , 3.7523682 ,
       3.3443651 , 3.8638697 , 4.63544   , 5.9941187 , 5.066087  ,
       5.491786  , 3.8712306 , 5.0054817 , 3.5633752 , 3.176874  ,
       3.7344499 , 3.8388608 , 3.971253  ], dtype=float32)

In [49]:
times_true = []
event_true = []
for idx, rows in y_test.iterrows():
    if rows['Survival_label_upper_bound'] == np.inf or not rows['Survival_label_upper_bound']:
        event = 0
    else:
        event = 1 
    event_true.append(event)
    times = rows['Survival_label_lower_bound']
    times_true.append(times)
times_true = np.array(times_true)
event_true = np.array(event_true)

In [50]:
from lifelines.utils import concordance_index
c_index = concordance_index(times_true, times_pred, event_true)
c_index

0.6181318681318682

# Support2

In [75]:
df = pd.read_csv('../data/support2.csv')

In [76]:
data = SupportDataset(df)

/mnt/c/Users/rizki/Master Files/ThesisUMCG/Survival-Tree-Analysis/src/dataset.py:95: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.data.replace({False: 0, True: 1}, inplace=True)


In [77]:

d = data.get_data()
d

array([[62.84998,  0.     ,  1.     , ...,  1.     ,  0.     ,  0.     ],
       [60.33899,  2.     ,  3.     , ...,  0.     ,  1.     ,  0.     ],
       [52.74698,  2.     ,  4.     , ...,  0.     ,  1.     ,  0.     ],
       ...,
       [70.38196,  1.     , 18.     , ...,  0.     ,  1.     ,  0.     ],
       [47.01999,  1.     , 22.     , ...,  0.     ,  0.     ,  1.     ],
       [81.53894,  1.     ,  1.     , ...,  0.     ,  1.     ,  0.     ]])

In [78]:
X_train, X_test, y_train, y_test = data.get_train_test_xgboost()

In [79]:
dtrain = xgb.DMatrix(X_train)
dtrain.set_float_info('label_lower_bound', y_train['Survival_label_lower_bound'])
dtrain.set_float_info('label_upper_bound', y_train['Survival_label_upper_bound'])|

SyntaxError: invalid syntax (3802997179.py, line 3)

In [80]:

dvalid = xgb.DMatrix(X_test)
dvalid.set_float_info('label_lower_bound', y_test['Survival_label_lower_bound'])
dvalid.set_float_info('label_upper_bound', y_test['Survival_label_upper_bound'])

In [81]:
params = {'verbosity': 0,
          'objective': 'survival:aft',
          'eval_metric': 'aft-nloglik',
          'tree_method': 'hist',
          'learning_rate': 0.05,
          'aft_loss_distribution': 'normal',
          'aft_loss_distribution_scale': 1.2,
          'max_depth': 6,
          'lambda': 0.01,
          'alpha': 0.02}

bst = xgb.train(params, dtrain, num_boost_round=1000,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                early_stopping_rounds=10)

[0]	train-aft-nloglik:15.00177	valid-aft-nloglik:15.30634
[1]	train-aft-nloglik:14.05325	valid-aft-nloglik:14.35721
[2]	train-aft-nloglik:13.10038	valid-aft-nloglik:13.41016


[3]	train-aft-nloglik:12.18803	valid-aft-nloglik:12.49071
[4]	train-aft-nloglik:11.35780	valid-aft-nloglik:11.65341
[5]	train-aft-nloglik:10.60332	valid-aft-nloglik:10.89063
[6]	train-aft-nloglik:9.91694	valid-aft-nloglik:10.19749
[7]	train-aft-nloglik:9.29235	valid-aft-nloglik:9.56679
[8]	train-aft-nloglik:8.72220	valid-aft-nloglik:8.98999
[9]	train-aft-nloglik:8.20292	valid-aft-nloglik:8.46526
[10]	train-aft-nloglik:7.72934	valid-aft-nloglik:7.98467
[11]	train-aft-nloglik:7.29714	valid-aft-nloglik:7.54617
[12]	train-aft-nloglik:6.90242	valid-aft-nloglik:7.14511
[13]	train-aft-nloglik:6.54185	valid-aft-nloglik:6.77727
[14]	train-aft-nloglik:6.21188	valid-aft-nloglik:6.44134
[15]	train-aft-nloglik:5.91021	valid-aft-nloglik:6.13329
[16]	train-aft-nloglik:5.63362	valid-aft-nloglik:5.85188
[17]	train-aft-nloglik:5.38037	valid-aft-nloglik:5.59375
[18]	train-aft-nloglik:5.14820	valid-aft-nloglik:5.35673
[19]	train-aft-nloglik:4.93527	valid-aft-nloglik:5.13936
[20]	train-aft-nloglik:4.74008	

In [82]:
times_pred = bst.predict(dvalid, output_margin=True)
times_pred

array([6.733583 , 6.653086 , 7.761915 , ..., 7.026075 , 7.7265673,
       7.647699 ], dtype=float32)

In [83]:
times_true = []
event_true = []
for idx, rows in y_test.iterrows():
    if rows['Survival_label_upper_bound'] == np.inf or not rows['Survival_label_upper_bound']:
        event = 0
    else:
        event = 1 
    event_true.append(event)
    times = rows['Survival_label_lower_bound']
    times_true.append(times)
times_true = np.array(times_true)
event_true = np.array(event_true)

In [84]:
y_test.head()

,Survival_label_lower_bound,Survival_label_upper_bound
8630,109,inf
1069,866,866.0
6549,128,inf
5459,35,inf
5318,582,582.0


In [85]:
from lifelines.utils import concordance_index
c_index = concordance_index(times_true, times_pred, event_true)
c_index

0.7553952338009053